In [1]:
import pandas as pd 
import re
import requests
import pickle

In [2]:
data = requests.get('https://www.ams.usda.gov/mnreports/jo_gr112.txt').text
pickle.dump(data,open('intra.pickle','wb'))

In [3]:
data=pickle.load(open('intra.pickle','rb'))

In [4]:
crops = [ 'Louisiana Morning bids and basis for US 2 Soft Red Winter Wheat',
         'Louisiana Morning bids and basis for US 2 Yellow Corn',
         'Louisiana Morning bids and basis for US 1 Yellow Soybeans',
         'Louisiana Morning bids and basis for US 2 Yellow Sorghum', 
         'Texas Morning bids and basis for US 1 Hard Red Winter Wheat', 
         'Texas Morning bids and basis for US 2 Yellow Sorghum']



r = re.compile(r'|'.join(crops))
reports = re.split(r,data) #splits based on crop
reps = [x.split('\r\n') for x in reports]
reps
# months = ('Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct', 'Nov', 'Dec') #months to search text by
reps

[['',
  'JO_GR112',
  'Portland, OR    Mon  Mar 18, 2019    USDA Market News',
  '',
  'Louisiana and Texas Export Bids',
  '',
  'Gulf Export bids and basis for grain delivered to Gulf export elevators,',
  'CIF barge to Louisiana Gulf (Mississippi River) and rail-truck Texas Gulf,',
  'delivery periods as specified, dollars per bushel, except sorghum per cwt.',
  'Bids as of 11:30 Central time; Subject to change.',
  '',
  '   In early trading bids were not fully established but were indicated as',
  'follows for Louisiana:  soft red winter wheat lower, yellow corn lower,',
  'soybeans were lower and yellow sorghum was not available. Texas cash bids',
  'were lower for hard red winter wheat and lower for yellow sorghum.',
  '',
  '',
  ''],
 ['',
  'Cash Bids                           Basis',
  'Mar=    5.4325-5.4825                +87K  to +92K',
  'Apr=    5.4125-5.4225                +85K  to +86K',
  'May=    5.3925-5.4225                +83K  to +86K',
  'Jun=    5.2825-5.3125  

In [5]:
list1= []
a = re.compile(r'^\w{,3}=')
for x in reps[:]:
    new = [ y for y in x if re.match(a,y) ]
    list1.append(new)
list1

def get_crop_type():
    
    '''This funciton gets the length of each section of parsed data, number of entries per crop type and creates a list
       that corresponds with the name of the crop for each entry to be used in normalizing the dataframe
    '''
    crop_count = []
    for item in list1:
        if item  !=[]:
            crop_count.append(len(item))
    crops_list = []
    for x in range(len(crop_count)):
        crops_list.append( [crops[x] for crop in range(0,crop_count[x])])

    crop_list_final = [item for sublist in crops_list for item in sublist]
    return crop_list_final


crop_type = get_crop_type()




In [6]:
list3= []
for x in list1:
    for y in x:
        list3.append(re.split(r'\s{2,}',y)) 
list3

[['Mar=', '5.4325-5.4825', '+87K', 'to +92K'],
 ['Apr=', '5.4125-5.4225', '+85K', 'to +86K'],
 ['May=', '5.3925-5.4225', '+83K', 'to +86K'],
 ['Jun=', '5.2825-5.3125', '+65N', 'to +68N'],
 ['Jul=', '5.2825-5.3125', '+65N', 'to +68N'],
 ['Aug=', 'NA', 'NA'],
 ['Mar=', '4.1775-4.2575', '+47K', 'to +55K'],
 ['Apr=', '4.1675-4.2375', '+46K', 'to +53K'],
 ['May=', '4.1675-4.2375', '+46K', 'to +53K'],
 ['Jun=', '4.2500', '+45N'],
 ['Jul=', '4.2500', '+45N'],
 ['Aug=', '4.2975', '+43U'],
 ['Mar=', '9.2325-9.5125', '+21K', 'to +49K'],
 ['Apr=', '9.2125-9.4525', '+19K', 'to +43K'],
 ['Jun=', '9.3475-9.5475', '+19N', 'to +39N'],
 ['Jul=', '9.3875-9.5475', '+23N', 'to +39N'],
 ['Aug=', 'NA', 'NA'],
 ['Mar=', 'NA', 'NA'],
 ['Mar=', '5.4800-5.6600', '+112K', 'to +130K'],
 ['Mar=', '7.6925', '+60K'],
 ['NC=', '7.2175', '+10Z']]

In [7]:
  
try:
    
    df3 = pd.DataFrame(list3, columns = ['Month','Cash_Bids', 'Basis1_og'])

except AssertionError:
    df3 = pd.DataFrame(list3,columns= ['Month','Cash_Bids', 'Basis1_og','Basis2_og'])
    # look for better regex to remove to and +-
    df3['Basis1_1'] = df3['Basis1_og'].str.strip('to +-').str[:-1]
    df3['Basis1_2'] = df3['Basis2_og'].str.strip('to +-').str[:-1]
    
    
df3['Crop'] = crop_type
df3['Date'] = pd.Timestamp.now()
df3['Cash_Bids_1'] = df3['Cash_Bids'].str.split('-', expand=True)[0]
df3['Cash_Bids_2'] = df3['Cash_Bids'].str.split('-', expand=True)[1]
df3['Contract'] = df3['Basis1_og'].str.replace('\d','') # fix here for no comp error-- need a regex that finds only word chars and fill with something 

df3['Month'] = df3['Month'].str.strip('=')

In [8]:
intra = df3[['Month','Cash_Bids_1','Cash_Bids_2','Basis1_1','Basis1_2','Contract','Crop','Date']]
intra = intra.fillna('NA')
intra = intra.replace('N', 'NA')


In [9]:
intra.head()

,Month,Cash_Bids_1,Cash_Bids_2,Basis1_1,Basis1_2,Contract,Crop,Date
0,Mar,5.4325,5.4825,87,92,+K,Louisiana Morning bids and basis for US 2 Soft...,2019-03-18 12:42:18.821440
1,Apr,5.4125,5.4225,85,86,+K,Louisiana Morning bids and basis for US 2 Soft...,2019-03-18 12:42:18.821440
2,May,5.3925,5.4225,83,86,+K,Louisiana Morning bids and basis for US 2 Soft...,2019-03-18 12:42:18.821440
3,Jun,5.2825,5.3125,65,68,+N,Louisiana Morning bids and basis for US 2 Soft...,2019-03-18 12:42:18.821440
4,Jul,5.2825,5.3125,65,68,+N,Louisiana Morning bids and basis for US 2 Soft...,2019-03-18 12:42:18.821440


In [10]:
pickle.dump(intra, open('jogr112_intraday_frame','wb'))